读取数据集

In [ ]:
library(tidyverse)
dt <- read.csv(file.path(getwd()%>%dirname(),"data/glue.csv"))
dt <- dt %>%
  arrange(Model)
dt

转变为宽数据集

In [ ]:
dt_wider1 <- dt %>%
                pivot_wider(
                  id_cols = "Model",
                  names_from = c("Task","Encoder","Year"),
                  values_from = "Score",
                  names_sep="_"
                )
dt_wider1

将宽数据集变为长数据集：
在变宽厚，有一些行的变量为NA，这是由于本身没有这个数据的原因
因此在恢复为长数据框时这些NA会被保留
因此需要na.omit掉

In [ ]:
dt_longer1 <- dt_wider1 %>%
                  pivot_longer(
                    cols = matches("_"),
                    names_to = c("Task","Encoder","Year"),
                    names_pattern = "(.*)_(.*)_(.*)", # pattern order is -> "Task","Encoder","Year"
                    values_to = "Score"
                  ) %>%
                  arrange(Model) %>%
                  na.omit()
dt_longer1

将Task中的各个变量储存为各自的列表

In [ ]:
dt_longer1_2 <- dt_wider1 %>%
                  pivot_longer(
                    cols = matches("_"),
                    names_to = c(".value","Encoder","Year"),
                    names_pattern = "(.*)_(.*)_(.*)"
                    # values_to = "Score" # 因为各个Task会作为列保存，Score作为value分布储存在各列中，因此无需明明Score的列名
                  ) %>%
                  arrange(Model) %>%
                  na.omit()
dt_longer1_2

只变宽两列

In [ ]:
dt_wider2 <- dt %>%
                pivot_wider(
                  id_cols = c("Model","Task"),
                  names_from = c("Encoder","Year"),
                  values_from = "Score",
                  names_sep="_"
                ) 
dt_wider2

将dt_wider2复原为长列表

In [ ]:
dt_longer2 <- dt_wider2 %>%
                  pivot_longer(
                    cols = matches("_"),
                    names_to = c("Encoder","Year"),
                    names_pattern = "(.*)_(.*)",
                    values_to = "Score"
                  ) %>%
                  arrange(Model) %>%
                  na.omit()
dt_longer2

将所有的NA替换为0

In [ ]:
dt_wider2_na.rm <- dt_wider2 %>%
                        mutate_all(~replace_na(.,0)) %>%
                        arrange(Model,Task)
dt_wider2_na.rm

生成一个2018与2019年的平均值的列

In [ ]:
dt_wider2_na.rm %>%
            select(-c("LSTM_2017","LSTM_2018")) %>%
            mutate(
              across(
                contains("_2018"), 
                 ~ (. + get(sub("_2018", "_2019", cur_column())))/2, 
                .names = "{.col}_mean")
              ) %>%
            rename_with(
              ~ gsub("_2018_mean", "_mean", .), 
              contains("_2018_mean")
              ) %>%
            arrange(Model,Task)

另一种生成平均值列的方法

In [ ]:
dt_wider2_na.rm_2 <- dt_wider2_na.rm %>%
                          select(-c(LSTM_2017, LSTM_2018)) %>%
                          pivot_longer(
                            cols = matches("_"),
                            names_to = c("Encode","Year"),
                            names_pattern = "(.*)_(.*)",
                            values_to = "Score"
                          ) 
dt_wider2_na.rm_2
dt_wider2_na.rm_2 %>%
  group_by(Model,Task,Encode) %>%
  summarize(
    mean_Score = mean(Score)
  ) 